PyTorch 의 nn.Module 에는 train / evaluation mode 가 있습니다. 

Dropout 은 일정한 비율로 특정 마디의 값을 0 으로 치환하는 기법입니다. 그리고 학습 시에는 dropout 을 적용하지만, 학습된 모델을 적용할 때에는 dropout 을 적용하지 않습니다. 그래서 마치 ensemble 과 같은 효과를 얻습니다. 이처럼 학습과 모델 적용 단계에 따라 Dropout layer 가 다르게 작동하도록 만들기 위하여 nn.Module 의 mode 를 변경해야 합니다.

우리는 `3 - 10 - 2` 의 hidden layer 를 지니는 Feed forward neural network 를 만듭니다. activation function 은 hyper tangent 를 이용합니다.

그리고 train 과 eval mode 에 따른 값의 차이를 살펴봅니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim

print(torch.__version__)

1.0.1.post2


In [2]:
class Net(nn.Module):
    def __init__(self, in_dim=3, hidden_dim=10, out_dim=2, dropout=0.5):
        super(Net, self).__init__()
        self.in_to_h1 = nn.Linear(in_dim, hidden_dim, bias=False)
        self.h1drop = nn.Dropout(dropout)
        self.h1_to_out = nn.Linear(hidden_dim, out_dim, bias=False)

    def forward(self, x, debug=False):
        h1 = torch.tanh(self.in_to_h1(x))
        drop = self.h1drop(h1)
        out = self.h1_to_out(drop)

        if debug:
            print('in_to_h1\n{}'.format(self.in_to_h1.weight.data))
            print('h1\n{}'.format(h1.data), end='\n\n')
            print('dropout\n{}'.format(drop.data), end='\n\n')
            print('out\n{}'.format(out.data), end='\n\n')
        return out

net = Net()
x = torch.rand((1, 3))

네트워크의 구조를 확인합니다.

In [3]:
print(net)

Net(
  (in_to_h1): Linear(in_features=3, out_features=10, bias=False)
  (h1drop): Dropout(p=0.5)
  (h1_to_out): Linear(in_features=10, out_features=2, bias=False)
)


테스트에 이용할 3 차원 input vector 입니다.

In [4]:
x

tensor([[0.4617, 0.2211, 0.0504]])

net 을 train mode 로 만듭니다. 함수를 실행하지 않아도 기본으로 train mode 입니다. 첫번째 hidden 을 거친 값은 [-1, 1] 사이의 10 차원 벡터입니다. Dropout layer 를 지나면서 일부가 0 으로 변하였습니다. 0 이 아닌 값이 h1 의 두 배가 되는 이유는 loss 의 크기를 유지하기 위하여 1 - dropout ratio 의 역수로 벡터의 크기를 키우기 때문입니다.

In [5]:
net.train()
z = net.forward(x, debug=True)

in_to_h1
tensor([[ 0.4802, -0.3685, -0.1947],
        [-0.1639,  0.1439,  0.1251],
        [-0.3477, -0.4726, -0.4202],
        [ 0.2857,  0.2486, -0.4135],
        [-0.3216, -0.1887,  0.1364],
        [ 0.3519, -0.3512, -0.4188],
        [ 0.2502, -0.1079, -0.4374],
        [ 0.4281, -0.5304, -0.1201],
        [ 0.2121, -0.1702,  0.1905],
        [ 0.4310,  0.5214, -0.1435]])
h1
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

dropout
tensor([[ 0.0000, -0.0751, -0.0000,  0.3290, -0.3626,  0.1272,  0.1390,  0.1483,
          0.0000,  0.5955]])

out
tensor([[-0.1116, -0.0689]])



같은 input vector 를 적용하면 10 차원의 다른 노드들이 0 으로 변합니다.

In [6]:
z = net.forward(x, debug=True)

in_to_h1
tensor([[ 0.4802, -0.3685, -0.1947],
        [-0.1639,  0.1439,  0.1251],
        [-0.3477, -0.4726, -0.4202],
        [ 0.2857,  0.2486, -0.4135],
        [-0.3216, -0.1887,  0.1364],
        [ 0.3519, -0.3512, -0.4188],
        [ 0.2502, -0.1079, -0.4374],
        [ 0.4281, -0.5304, -0.1201],
        [ 0.2121, -0.1702,  0.1905],
        [ 0.4310,  0.5214, -0.1435]])
h1
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

dropout
tensor([[0.0000, -0.0000, -0.0000, 0.0000, -0.0000, 0.1272, 0.1390, 0.0000, 0.1396,
         0.5955]])

out
tensor([[ 0.0461, -0.1098]])



하지만 eval mode 에서는 hidden vector 의 모든 값이 그대로 유지됨을 확인할 수 있습니다.

In [7]:
net.eval()
z = net.forward(x, debug=True)

in_to_h1
tensor([[ 0.4802, -0.3685, -0.1947],
        [-0.1639,  0.1439,  0.1251],
        [-0.3477, -0.4726, -0.4202],
        [ 0.2857,  0.2486, -0.4135],
        [-0.3216, -0.1887,  0.1364],
        [ 0.3519, -0.3512, -0.4188],
        [ 0.2502, -0.1079, -0.4374],
        [ 0.4281, -0.5304, -0.1201],
        [ 0.2121, -0.1702,  0.1905],
        [ 0.4310,  0.5214, -0.1435]])
h1
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

dropout
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

out
tensor([[ 0.0411, -0.1151]])



다시 train mode 로 변환하면 몇 개 마디 값이 0 으로 변함을 확인할 수 있습니다.

In [8]:
net.train()
z = net.forward(x, debug=True)

in_to_h1
tensor([[ 0.4802, -0.3685, -0.1947],
        [-0.1639,  0.1439,  0.1251],
        [-0.3477, -0.4726, -0.4202],
        [ 0.2857,  0.2486, -0.4135],
        [-0.3216, -0.1887,  0.1364],
        [ 0.3519, -0.3512, -0.4188],
        [ 0.2502, -0.1079, -0.4374],
        [ 0.4281, -0.5304, -0.1201],
        [ 0.2121, -0.1702,  0.1905],
        [ 0.4310,  0.5214, -0.1435]])
h1
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

dropout
tensor([[ 0.0000, -0.0000, -0.5573,  0.0000, -0.0000,  0.1272,  0.1390,  0.1483,
          0.1396,  0.5955]])

out
tensor([[ 0.1003, -0.2834]])



## torch IO (save / load)

### save whole model

In [9]:
model_path = 'sample_model.pt'
torch.save(net, model_path)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [10]:
loaded_net = torch.load(model_path)
loaded_net.eval()
loaded_net(x, debug=True)

in_to_h1
tensor([[ 0.4802, -0.3685, -0.1947],
        [-0.1639,  0.1439,  0.1251],
        [-0.3477, -0.4726, -0.4202],
        [ 0.2857,  0.2486, -0.4135],
        [-0.3216, -0.1887,  0.1364],
        [ 0.3519, -0.3512, -0.4188],
        [ 0.2502, -0.1079, -0.4374],
        [ 0.4281, -0.5304, -0.1201],
        [ 0.2121, -0.1702,  0.1905],
        [ 0.4310,  0.5214, -0.1435]])
h1
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

dropout
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

out
tensor([[ 0.0411, -0.1151]])



tensor([[ 0.0411, -0.1151]], grad_fn=<MmBackward>)

### save only parameters

In [11]:
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

in_to_h1.weight 	 torch.Size([10, 3])
h1_to_out.weight 	 torch.Size([2, 10])


In [12]:
model_path = 'sample_model_params.pt'
torch.save(net.state_dict(), model_path)

loaded_net = Net()
loaded_net.load_state_dict(torch.load(model_path))
loaded_net.eval()

Net(
  (in_to_h1): Linear(in_features=3, out_features=10, bias=False)
  (h1drop): Dropout(p=0.5)
  (h1_to_out): Linear(in_features=10, out_features=2, bias=False)
)

In [13]:
loaded_net(x, debug=True)

in_to_h1
tensor([[ 0.4802, -0.3685, -0.1947],
        [-0.1639,  0.1439,  0.1251],
        [-0.3477, -0.4726, -0.4202],
        [ 0.2857,  0.2486, -0.4135],
        [-0.3216, -0.1887,  0.1364],
        [ 0.3519, -0.3512, -0.4188],
        [ 0.2502, -0.1079, -0.4374],
        [ 0.4281, -0.5304, -0.1201],
        [ 0.2121, -0.1702,  0.1905],
        [ 0.4310,  0.5214, -0.1435]])
h1
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

dropout
tensor([[ 0.1296, -0.0375, -0.2786,  0.1645, -0.1813,  0.0636,  0.0695,  0.0741,
          0.0698,  0.2978]])

out
tensor([[ 0.0411, -0.1151]])



tensor([[ 0.0411, -0.1151]], grad_fn=<MmBackward>)